In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/montcoalert/911.csv')
df.head()

In [ ]:
df.info()

# Top five zip codes for 911 calls

In [ ]:
df['zip'].value_counts().head()

# Top 5 townships (twp) for 911 calls

In [ ]:
df['twp'].value_counts().head()

# Unique title columns

In [ ]:
df['title'].nunique()

 In the titles column there are "Reasons/Departments" specified before the title code. These are EMS, Fire, and Traffic. Use .apply() with a custom lambda expression to create a new column called "Reason" that contains this string value
 
 
 
 For example, if the title column value is EMS: BACK PAINS/INJURY , the Reason column value would be EMS. 

In [ ]:
df['Reason']=df['title'].apply(lambda title:title.split(':')[0])

 Most common Reason for a 911 call based off of this new column

In [ ]:
df['Reason'].value_counts()

 Use seaborn to create a countplot of 911 calls by Reason.

In [ ]:
sns.countplot(x='Reason',data=df,palette='viridis')

What is the data type of the objects in the timeStamp column?

In [ ]:
type(df['timeStamp'].iloc[0])

Use pd.to_datetime to convert the column from strings to DateTime objects

In [ ]:
df['timeStamp']=pd.to_datetime(df['timeStamp'])


Use .apply() to create 3 new columns called Hour, Month, and Day of Week. You will create these columns based off of the timeStamp column, reference the solutions if you get stuck on this step.

In [ ]:
df['Hour']=df['timeStamp'].apply(lambda time: time.hour)
df['Month']=df['timeStamp'].apply(lambda time: time.month)
df['Day of week']=df['timeStamp'].apply(lambda time: time.dayofweek)

Use the .map() with this dictionary to map the actual string names to the day of the week: 

dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
df['Day of week']=df['Day of week'].map(dmap)

Use seaborn to create a countplot of the Day of Week column with the hue based off of the Reason column. 

In [ ]:
sns.countplot(x='Day of week',data=df,hue='Reason')
plt.legend(bbox_to_anchor=(1,1),loc=2)

Same for Month

In [ ]:
sns.countplot(x='Month',data=df,hue='Reason')
plt.legend(bbox_to_anchor=(1,1),loc=2)

Same for Hour

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='Hour',data=df,hue='Reason')
plt.legend(bbox_to_anchor=(1,1),loc=2)

Did you notice something strange about the Plot? 

 It is missing some months! 9,10, and 11 are not there.
 You should have noticed it was missing some Months, let's see if we can maybe fill in this information by plotting the information in another way, possibly a simple line plot that fills in the missing months, in order to do this, we'll need to do some work with pandas

 Now create a gropuby object called byMonth, where you group the DataFrame by the month column and use the count() method for aggregation. Use the head() method on this returned DataFrame.

In [ ]:
byM=df.groupby('Month').count()
byM.head()

 Now create a simple plot off of the dataframe indicating the count of calls per month

In [ ]:
#could be any column
byM['twp'].plot()

Now see if you can use seaborn's lmplot() to create a linear fit on the number of calls per month. Keep in mind you may need to reset the index to a column.

In [ ]:
data=byM.reset_index()
data

In [ ]:
sns.lmplot(x='Month',y='twp',data=data)

Create a new column called 'Date' that contains the date from the timeStamp column. You'll need to use apply along with the .date() method

In [ ]:
df['Date']=df['timeStamp'].apply(lambda t:t.date())

In [ ]:
plt.figure(figsize=(12,6))
df.groupby('Date').count()['twp'].plot()
plt.tight_layout()

Now recreate this plot but create 3 separate plots with each plot representing a Reason for the 911 call

In [ ]:
plt.figure(figsize=(12,6))
df[df['Reason']=='Traffic'].groupby('Date').count()['twp'].plot()
plt.title('Traffic')
plt.tight_layout

In [ ]:
plt.figure(figsize=(12,6))
df[df['Reason']=='EMS'].groupby('Date').count()['twp'].plot()
plt.title('EMS')
plt.tight_layout

In [ ]:
plt.figure(figsize=(12,6))
df[df['Reason']=='Fire'].groupby('Date').count()['twp'].plot()
plt.title('EMS')
plt.tight_layout

 Now let's move on to creating heatmaps with seaborn and our data. We'll first need to restructure the dataframe so that the columns become the Hours and the Index becomes the Day of the Week. There are lots of ways to do this, but I would recommend trying to combine groupby with an unstack method. Reference the solutions if you get stuck on this!

In [ ]:
dayHour = df.groupby(by=['Day of week','Hour']).count()['Reason'].unstack()
dayHour.head()

Now create a HeatMap using this new DataFrame.

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(dayHour,cmap='magma')

Now create a clustermap using this DataFrame.

In [ ]:
sns.clustermap(dayHour,cmap='cividis')

 Now repeat these same plots and operations, for a DataFrame that shows the Month as the column. 

In [ ]:
dayMonth = df.groupby(by=['Day of week','Month']).count()['Reason'].unstack()
dayMonth.head()

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(dayMonth,cmap='plasma')

In [ ]:
sns.clustermap(dayMonth,cmap='viridis')